In [ ]:
options(warn = -1)
library(pracma)
library(tibble)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(tidyr)
library(stringr)
library(ggthemes)
library(mxnet)

In [ ]:
load("mnist/mnist_data.RData")
mnist_data_mat <- data.matrix(mnist_data)

In [ ]:
head(mnist_data)

# Machine Learning using Neural Networks (Part 2)

Welcome to the tutorial on Machine Learning using Neural Networks! In this part of the tutorial, we will use neural networks to recognize handwritten digits. Will will start by outlining why neural networks are good for recognizing images, and then train a neural network to recognize handwritten digits. We will use a famous dataset for this task, which is called MNIST.

Let's get started!

## Image Recognition using Neural Networks

How can computer programs recognize objects in a photograph? This has been one of the central questions in the computer vision research. It is very simple for humans to recognize objects in the photographs, just like we do in real world. But for computers, this task becomes notoriously difficult! Although with a little help from neural networks, it becomes rather easy because neural networks behave in a slightly intuitive way. Here is how a neural network would recognize whether there is a face in an image or not:

<img src="mnist/imgrecog_1.png"/>

What you see in the image above essentially tell us that neural networks try to recognize different 'features' of the image first, and based on appearance of those features, they decide whether the image contains the signal of interest or not (a human face in this case).

The problem of detecting a face in a photograph is a binary classification problem, but for our example, we are going to turn to a multi-class classification problem.

## Example: Recognizing Handwritten Digits using Neural Networks

Okay! We're in the final section of our workshop. In this section, we will try train a neural network model that can tell us which handwritten digit is there in the image we give to it. We are going to use a database of images that was originally developed in 1994 called the MNIST dataset. It is based on samples of high school students and U. S. Census Bureau employees' writing- https://en.wikipedia.org/wiki/MNIST_database

<img src="mnist/mnist_1.png"/>

You can see that there are many different ways of wriring 0s, 1s, 2s, and so on. How is neural network going to detect which digit is there in the image? It turns out that neural network will try to break down the problem in parts and work it out.

### Detecting a 0

Let us say the neural network is trying to detect whether there is 0 in the image or not. It will proceed like this:

- Detect handwriting strokes in different parts of the image
- Put these strokes together to figure out what digit is there in the image

For a 0, we might have these strokes in the image:

<img src="mnist/mnist_2.png" width=350/>

where the final image looks like this:

<img src="mnist/mnist_3.png" width=105/>

And remember, there can be many different types of 0s! But most of them will have circular strokes in all four sides of the image.

The code below plots images that were labled (the training data) as zeros.

In [ ]:
zero_pos <- which(mnist_data$label == 0)

options(repr.plot.width = 8, repr.plot.height = 10)

mnist_data %>%
  slice(zero_pos[1:100]) %>%
  mutate(example = row_number()) %>%
  gather(key = pixnum, value = inten, -example, -label) %>%
  mutate(pixnum = as.integer(str_replace(pixnum, "pixel", "")) + 1,
         x = factor(floor(pixnum / 28) + 1, 28:1),
         y = factor(pixnum %% 28 + 1, 1:28)) %>%
  drop_na() %>%
  ggplot(aes(y, x)) +
  geom_tile(aes(fill = inten)) +
  facet_wrap(~ example, nrow = 10) +
  scale_fill_continuous(low = "white", high = "black") +
  theme_minimal() +
  theme(legend.position = "none",
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        axis.title = element_blank()) +
  ggtitle("100 different handwritten 0s from MNIST dataset")

### Detecting 0 to 9

So you now have a rough idea about how can a neural network detect 0 in an image. However, we need to detect all the digits, from 0 to 9. How will we do this using using a neural network?

So far, we have only talked about binary outcomes where a neural network is detecting whether there is 'some thing' present in its input or not. This requires us to have one neuron in the output layer. But if we want to detect multiple things, we need to have multiple neurons in the output layer, one for each thing we need to detect. In our case, we need to detect all the arabic numerals, so there will be 10 output nodes. Here is what our neural network will look like:

<img src="mnist/mnist_4.png"/>

Let us find out where those 784 input neurons are coming from.

### MNIST Dataset

The MNIST dataset contains many thousand training examples for handwritten digits. Each training example is a 28x28 pixel image. This means that we have 28x28 = 784 features for each of our training example. As the output, we have a numeric value from 0 to 9, which tells the actual digit that is stored in those 784 pixels. We are going to use this information to train and evalulate our model.

Before we jump further in, you might ask, what do these 784 pixels contain? They contain what is called pixel intensity value, a value between 0 and 255 which indicates how dark the pixel is. If we print these pixel intensity values, we get to see the actual image. Here is an example:

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)

mnist_data %>%
  slice(66) %>%
  mutate(example = row_number()) %>%
  gather(key = pixnum, value = inten, -example, -label) %>%
  mutate(pixnum = as.integer(str_replace(pixnum, "pixel", "")) + 1,
         x = factor(floor(pixnum / 28) + 1, 28:1),
         y = factor(pixnum %% 28 + 1, 1:28)) %>%
  drop_na() %>%
  ggplot(aes(y, x)) +
  geom_tile(aes(fill = inten, width = .8, height = .8), size = 1, alpha = .6) +
  geom_text(aes(label = inten), size = 1.8, color = "black") +
  scale_fill_continuous(low = "white", high = "black") +
  theme(axis.ticks = element_blank(),
        axis.text = element_blank(),
        axis.title = element_blank()) +
  labs(fill = "Intensity value", title = "A digit 1 with pixel intensities from MNIST dataset")

We are going to feed a lot of these images to our neural network, and it will learn characteristics of different digits for us. Then we can use this trained model to make predictions.

### Model Training

Let us start the model training by looking at the the dataset. We will use <b>mxnet</b> (https://github.com/Xilinx/mxnet/tree/master/R-package) package for our machine learning task. Here are some rows from our dataset:

In [ ]:
head(mnist_data_mat, 10)

Now we will divide the data in <b>training set</b> and <b>test set</b>. Our training set will have 35,000 examples and our test set will have 7,000 examples.

In [ ]:
head(test)

In [ ]:
test <- mnist_data_mat[1:7000,]
train <- mnist_data_mat[7001:42000,]

Next, we will organize the data into objects representing the x and y variables.

In [ ]:
train.x <- train[,-1]
train.y <- train[,1]
train.x <- t(train.x/255)

test.x <- test[,-1]
test.y <- test[,1]
test.x <- t(test.x/255)

cat("Training data dimensions")
dim(train.x)
table(train.y)

cat("Test data dimensions")
dim(test.x)
table(test.y)

Now we will configure our neural network.

In [ ]:
# Input layer
data <- mx.symbol.Variable("data")

# Hidden layer with 15 neurons
fc1 <- mx.symbol.FullyConnected(data, name="fc1", num_hidden=15)
act1 <- mx.symbol.Activation(fc1, name="tanh1", act_type="tanh")

# Output layer
fc2 <- mx.symbol.FullyConnected(act1, name="fc2", num_hidden=10)
softmax <- mx.symbol.SoftmaxOutput(fc2, name="sm")

Next we set the hyperparameters of the model:

In [ ]:
learning_rate <- 0.02
max_iterations <- 1

Now it is time to train the model!

In [ ]:
devices <- mx.cpu()
mx.set.seed(0)

model <- mx.model.FeedForward.create(softmax, X=train.x, y=train.y,
                                     ctx=devices, num.round=max_iterations, array.batch.size=100,
                                     learning.rate=learning_rate, momentum=0.9,  eval.metric=mx.metric.accuracy,
                                     initializer=mx.init.uniform(0.07),
                                     epoch.end.callback=mx.callback.log.train.metric(100))

Now that our model is trained, we will generate some predictions and assess the accuracy. <b>Remember, always assess accuracy using test data!</b>

### Model Evaluation

In [ ]:
preds <- predict(model, test.x)
dim(preds) # this is the shape of the matrix

We can look at the distribution of the answers in a comparison matrix.

In [ ]:
pred.label <- max.col(t(preds)) - 1
table(pred = pred.label, orig = test.y)

Look at the missed predictions.  Do you see a pattern?

Let us calculate the test accuracy of the model.

In [ ]:
pct_accuracy <- mean(pred.label == test.y)
cat("Test accuracy of the model is", pct_accuracy * 100, "%\n")

Not bad. How can we improve this model? There are different ways to improve predictions of a neural network:

- <b>Change architecture</b>
- <b>Try different hyperparameters</b>
- <b>Try advanced methods like convolution</b>

Trying these different things is out of the scope of this tutorial, but if you want to explore these options, you can do so in <a href="https://playground.tensorflow.org">TensorFlow Playground</a>.

## Conclusion

In this part of the tutorial, we saw how neural networks are trained using software packages. Don't worry if this was too complicated, you always get help about doing complex things when working in teams. Websites like [StackOverflow](https://stackoverflow.com/) can also help in case you get stuck.

We hope that you found both part 1 and part 2 of the tutorial useful.

## References

We would like to thank Michael Nielson for his excellent online textbook on neural networks. We extensively borrowed part 1 materials from his textbook which can be found <a href="http://neuralnetworksanddeeplearning.com">here</a>.

For part 2, we used the mxnet example presented <a href="https://mxnet.incubator.apache.org/tutorials/r/mnistCompetition.html">here</a>, but modified it according to our neural network architecture.